In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import sklearn as sk

In [65]:
from pandas import json_normalize

In [66]:
import json

file_path = 'result.json'


with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)


df = json_normalize(
    data, 
    record_path='text_entities',
    meta=['id', 'type', 'date', 'date_unixtime', 'actor', 'actor_id', 'action', 'inviter', 'text'],
    record_prefix='entity_',
    errors='ignore'
)


In [67]:
df.shape

(18005, 15)

In [68]:
df = df[df['entity_text'].notna() & (df['entity_text'] != '')]

In [69]:
df = df[["entity_text", "date", "text"]]

 - сколько спама
 - сколько объявлений по объявлению
 - сколько объявлений по забоу
 - в скольких говорится что забрали
 - в скольких объявлениях упоминается срок годности
 - в скольких упоминается корпус

In [70]:
df

,entity_text,date,text
0,Никто половину арбуза забрать не хочет?,2021-07-28T13:54:41,Никто половину арбуза забрать не хочет?
1,UPD: забрали,2021-07-28T13:58:50,UPD: забрали
2,какой корпус?,2021-07-30T16:13:40,какой корпус?
3,"Молодые и красивые, буду рада отдать в добрые ...",2021-08-03T13:29:33,"Молодые и красивые, буду рада отдать в добрые ..."
4,"1к, уехала, оставила у двери страчателлу и ост...",2021-08-05T17:39:14,"1к, уехала, оставила у двери страчателлу и ост..."
...,...,...,...
18000,сито большое бронь,2025-09-20T09:02:20,сито большое бронь
18001,сито поменбше бронь,2025-09-20T09:05:58,сито поменбше бронь
18002,Венчик бронь,2025-09-20T09:08:36,Венчик бронь
18003,друзья мб кому нибудь нужны банки жалко выкиды...,2025-09-20T09:19:20,друзья мб кому нибудь нужны банки жалко выкиды...


In [71]:
df["keywords"] = ""

In [72]:
import re
import pandas as pd

def clean_message(row):
    raw_text = row.get("text", "")
    if not isinstance(raw_text, str):
        raw_text = str(raw_text)

    raw_entity = row.get("entity_text", "")
    if not isinstance(raw_entity, str):
        raw_entity = str(raw_entity)


    cleaned_text = re.sub(r'[^\w\s]', ' ', raw_text).lower().strip()
    cleaned_entity = re.sub(r'[^\w\s]', ' ', raw_entity).lower().strip()


    row["text"] = cleaned_text
    row["entity_text"] = cleaned_entity
    row["keywords"] = cleaned_text.split()

    return row

In [73]:
df = df.apply(clean_message, axis=1)

In [74]:
df

,entity_text,date,text,keywords
0,никто половину арбуза забрать не хочет,2021-07-28T13:54:41,никто половину арбуза забрать не хочет,"[никто, половину, арбуза, забрать, не, хочет]"
1,upd забрали,2021-07-28T13:58:50,upd забрали,"[upd, забрали]"
2,какой корпус,2021-07-30T16:13:40,какой корпус,"[какой, корпус]"
3,молодые и красивые буду рада отдать в добрые ...,2021-08-03T13:29:33,молодые и красивые буду рада отдать в добрые ...,"[молодые, и, красивые, буду, рада, отдать, в, ..."
4,1к уехала оставила у двери страчателлу и ост...,2021-08-05T17:39:14,1к уехала оставила у двери страчателлу и ост...,"[1к, уехала, оставила, у, двери, страчателлу, ..."
...,...,...,...,...
18000,сито большое бронь,2025-09-20T09:02:20,сито большое бронь,"[сито, большое, бронь]"
18001,сито поменбше бронь,2025-09-20T09:05:58,сито поменбше бронь,"[сито, поменбше, бронь]"
18002,венчик бронь,2025-09-20T09:08:36,венчик бронь,"[венчик, бронь]"
18003,друзья мб кому нибудь нужны банки жалко выкиды...,2025-09-20T09:19:20,друзья мб кому нибудь нужны банки жалко выкиды...,"[друзья, мб, кому, нибудь, нужны, банки, жалко..."


In [75]:
df.shape

(17841, 4)

In [76]:

keywords_sell = set([
    "отдам", "отдаю", "предложу", "предлагаю", "поделюсь", "раздам", "раздаю",
    "есть лишнее", "осталось", "досталось", "в наличии", "заберите", "угощу",
    "оставила", "оставлю", "у двери", "возьмите", "кому надо", "кому нужно",
    "кому", "все еще есть", "осталась", "отдал", "отдалась", "отдала",
    "upd", "апдейт", "апд", "бесплатно", "даром", "за просто так", "безвозмездно",
    "передам", "продам", "продаю", "бронь", "бронь за кем", "забронировано",
    "поставила", "положила", "в контейнере", "в подъезде", "под дверью"
])

keywords_buy = set([
    "заберу", "возьму", "взял бы", "беру", "готов взять", "есть ли", "ищу",
    "можно", "можно мне", "кому не нужно", "пригодится", "нужен", "нужна", "нужно",
    "хочу", "можно забрать", "подскажите", "осталось ли", "забрал бы", "ищется",
    "было бы здорово", "приеду", "забегу", "никто не отдаёт", "не осталось ли", 
    "бронь", "бронирую", "забронирую"
])

keywords_age = set(["срок", "давно", "вчерашнее", "недавнее", "свежее", "просрочено", "просрочка"])
keywords_house = set(["корпус", "к", "1к", "2к", "3к", "парадная", "подъезд", "этаж", "дверь"])
keywords_already_taken = set(["уже", "забрали", "отбой", "нету", "съели", "забронировано", "забрали уже"])



def has_any(tokens, keyword_set):
    return any(token in keyword_set for token in tokens)


df["is_sell"] = df["keywords"].apply(lambda tokens: has_any(tokens, keywords_sell))
df["is_buy"] = df["keywords"].apply(lambda tokens: has_any(tokens, keywords_buy))
df["is_age_related"] = df["keywords"].apply(lambda tokens: has_any(tokens, keywords_age))
df["is_house_related"] = df["keywords"].apply(lambda tokens: has_any(tokens, keywords_house))
df["is_already_taken"] = df["keywords"].apply(lambda tokens: has_any(tokens, keywords_already_taken))

posts_sell = df[df["is_sell"]]
posts_buy = df[df["is_buy"]]
posts_age = df[df["is_age_related"]]
posts_house = df[df["is_house_related"]]
posts_already_taken = df[df["is_already_taken"]]

In [77]:
posts_sell.shape


(3113, 9)

In [78]:
posts_buy.shape

(3449, 9)

In [79]:
posts_age.shape

(224, 9)

In [80]:
posts_already_taken.shape

(1163, 9)

In [81]:
posts_house.shape

(3460, 9)

# Результат

- изначально сообщений: 26000
- мусор: 7000
- текстовые: 19000
- удаленные (в больше степени спам и не релевантное): 1200
- не удаленные: 17800
- продажа: 3113
- покупка: 3449
- упоминался срок годности: 224
- уже забрали: 1163
- уточнение корпуса: 3460
- забыл написать корпус: 147